<a href="https://colab.research.google.com/github/Taeseong-eom/ApplePython/blob/main/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 809M/814M [00:09<00:00, 75.7MB/s]
100% 814M/814M [00:09<00:00, 87.1MB/s]
unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [4]:
!unzip -q dogs-vs-cats-redux-kernels-edition.zip -d .

In [5]:
!unzip -q train.zip -d .

In [7]:
import os
import tensorflow as tf
import shutil

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/cat/' + i )
  if 'dog' in i:
    shutil.copyfile( '/content/train/' + i , '/content/dataset/dog/' + i )

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [8]:
import requests
# 전이학습을 위해선 모델 + w 가 필요하니
url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' # 구글의 w값들을 가져오자
r = requests.get(url, allow_redirects=True)

open('inception_v3.h5', 'wb').write(r.content)

87910968

In [12]:
from tensorflow.keras.applications.inception_v3 import InceptionV3 # 구글 인셉션 모델 가져오자

inception_model = InceptionV3(input_shape=(150,150,3), include_top=False, weights=None) # 원래는 (299,299,3) 이었는데 input_shape 를 수정.//include_top=마지막 출력층 가져올건지// weights=w 가져올건지
inception_model.load_weights('inception_v3.h5')

# inception_model.summary()

# 많은 레이어가 있지만
# 개, 고양이 분류만 할 수 있으면 되니 마지막에 내 레이어를 하나 추가하자.
# 단, 다른 레이어의 w를 건들면 안된다.
for i in inception_model.layers:
  i.trainable=False

# 레이어를 아주 살짝 건들여서 더 좋게 만들 수 있다. fine tuning
# unfreeze = Flase
# for i in inception_model.layers:
#   if i.name == 'mixed6': # 이렇게 하나씩 true로 바꿀 수 있고
#     i.trainable=True
#   # mixed6 부터 전부 바꾸고 싶으면
#   if i.name == 'mixed6':
#     unfreeze = True
#   if unfreeze == True:
#     i.trainable=True

마지막레이어 = inception_model.get_layer('mixed7')
print(마지막레이어)

In [13]:
import tensorflow as tf

layer1 = tf.keras.layers.Flatten()(마지막레이어.output) # 레이어 연결
layer2 = tf.keras.layers.Dense(1024, activation='relu')(layer1)
drop1=tf.keras.layers.Dropout(0.2)(layer2)
layer3 = tf.keras.layers.Dense(1, activation='sigmoid')(drop1)

model=tf.keras.Model(inception_model.input , layer3)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['acc'])
# 그리고 파인튜닝하고 싶으면 아래처럼 하고 러닝레이트를 아주 작게 설정한다.
# model.compile(loss='binary_crossentropy',optimizer='tf.keras.optimizer.Adam(lr=0.000001)', metrics=['acc'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

Epoch 1/2
313/313 [==============================] - 1260s 4s/step - loss: 0.1596 - acc: 0.9531 - val_loss: 0.0790 - val_acc: 0.9670
Epoch 2/2
313/313 [==============================] - 1217s 4s/step - loss: 0.0384 - acc: 0.9860 - val_loss: 0.0910 - val_acc: 0.9706
